# Machine Learning Engineer Nanodegree 2020
## Capstone Project
Ashish kumar<br>February 18, 2020
## Time Series Analysis of Air Quality Data

In [1]:
# import libraries
import pandas as pd
import numpy as np
import os
import math
import glob


## Selecting Best Dataset

The data set includes hourly air pollutants data from 12 nationally-controlled air-quality monitoring sites. So we can choose any of the given monitoring site dataset to work upon. <br><br> Instead of randomly picking any dataset, It is better to look for the null values. Lesser the null values, better will be the predictions.<br><br> Let us build Helper function for finding the best dataset with respect to null values. The idea is to compare the average of row-wise average null values of each multi-site data file

In [2]:
data_path = 'beijing-multisite-airquality-data-set'
# storing names of all datafiles
file_names = [file_name[len(data_path)+1:] for file_name in glob.glob(data_path + '/*.csv')]
for file_name in file_names: print(file_name)

PRSA_Data_Aotizhongxin_20130301-20170228.csv
PRSA_Data_Changping_20130301-20170228.csv
PRSA_Data_Dingling_20130301-20170228.csv
PRSA_Data_Dongsi_20130301-20170228.csv
PRSA_Data_Guanyuan_20130301-20170228.csv
PRSA_Data_Gucheng_20130301-20170228.csv
PRSA_Data_Huairou_20130301-20170228.csv
PRSA_Data_Nongzhanguan_20130301-20170228.csv
PRSA_Data_Shunyi_20130301-20170228.csv
PRSA_Data_Tiantan_20130301-20170228.csv
PRSA_Data_Wanliu_20130301-20170228.csv
PRSA_Data_Wanshouxigong_20130301-20170228.csv


In [3]:
from shutil import copyfile

def best_data_file(data_files, cache_file = ''):
    
    # create directory for storing the dataset
    if not os.path.exists('dataset'):
        os.mkdir('dataset')
    
    if os.listdir('dataset'):
        # read from the cache file
        cache_file = os.listdir('dataset')[0]
        print('Reading from cache file: dataset/',cache_file, sep='')
        return cache_file
    
    # Do the heavy lifting
    best_file = ''
    Min_average = math.inf 
    for file_name in data_files:
        # Reading each datafile in dataframe
        df = pd.read_csv(data_path+'/'+file_name)
        
        # overall average = average of (average of null value col-wise )
        overall_average = df.isna().mean().mean()
        if overall_average < Min_average:
            Min_average = overall_average
            best_file = file_name
        # print(file_name)
        # print(overall_average)
        
        # for visualizing row-wise percentages of null values
        # print(df.isna().mean().round(4)*100)
    
    # copy the best file in the dataset folder
    src = os.path.join(data_path,best_file)
    dst = os.path.join('dataset',best_file)
    copyfile(src,dst)
    print('path created: '+dst)
    return best_file

In [4]:
# test the function
best_file = best_data_file(file_names)

Reading from cache file: dataset/PRSA_Data_Nongzhanguan_20130301-20170228.csv


## Data loading and Preprocesing
#### 1) Loading Data

In [5]:
# load data
df = pd.read_csv(os.path.join(data_path,best_file))
df.head()

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,5.0,14.0,4.0,12.0,200.0,85.0,-0.5,1024.5,-21.4,0.0,NNW,5.7,Nongzhanguan
1,2,2013,3,1,1,8.0,12.0,6.0,14.0,200.0,84.0,-0.7,1025.1,-22.1,0.0,NW,3.9,Nongzhanguan
2,3,2013,3,1,2,3.0,6.0,5.0,14.0,200.0,83.0,-1.2,1025.3,-24.6,0.0,NNW,5.3,Nongzhanguan
3,4,2013,3,1,3,5.0,5.0,5.0,14.0,200.0,84.0,-1.4,1026.2,-25.5,0.0,N,4.9,Nongzhanguan
4,5,2013,3,1,4,5.0,5.0,6.0,21.0,200.0,77.0,-1.9,1027.1,-24.5,0.0,NNW,3.2,Nongzhanguan


#### 2) Indexing Data

For time series analysis, Our data should be indexed as Date-Time. For this we will be using pandas DateTimeIndex function. It is important to note that the Time period of our data is from March 1st, 2013 to February 28th, 2017. Lets verify it before proceeding.

In [22]:
# 1st row
df[:1]

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
0,1,2013,3,1,0,5.0,14.0,4.0,12.0,200.0,85.0,-0.5,1024.5,-21.4,0.0,NNW,5.7,Nongzhanguan


In [24]:
# last row
df[df.shape[0]-1:]

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
35063,35064,2017,2,28,23,10.0,28.0,7.0,48.0,600.0,39.0,8.6,1014.1,-15.9,0.0,NNE,1.3,Nongzhanguan


In [25]:
# check data info
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35064 entries, 0 to 35063
Data columns (total 18 columns):
No         35064 non-null int64
year       35064 non-null int64
month      35064 non-null int64
day        35064 non-null int64
hour       35064 non-null int64
PM2.5      34436 non-null float64
PM10       34624 non-null float64
SO2        34618 non-null float64
NO2        34372 non-null float64
CO         33858 non-null float64
O3         34558 non-null float64
TEMP       35044 non-null float64
PRES       35044 non-null float64
DEWP       35044 non-null float64
RAIN       35044 non-null float64
wd         34986 non-null object
WSPM       35050 non-null float64
station    35064 non-null object
dtypes: float64(11), int64(5), object(2)
memory usage: 4.8+ MB


Since the columns year, month, day and hour didn't contain any null value, So we can use pandas DateTimeIndex function with start date-time as 2013-03-01 00:00:00 and end date-time as 2017-02-28 23:00:00. As our dataset contains hourly data so we can use freq as 'H' which stands for Hour.

In [38]:
# creating Date-Time index column
df['Date-Time'] = pd.DatetimeIndex(start = '2013-03-01 00:00:00',end = '2017-02-28 23:00:00',freq ='H')
# setting index as Date-Time
df.set_index('Date-Time', drop = True, inplace = True)

In [39]:
df.head()

,No,year,month,day,hour,PM2.5,PM10,SO2,NO2,CO,O3,TEMP,PRES,DEWP,RAIN,wd,WSPM,station
Date-Time,,,,,,,,,,,,,,,,,,
2013-03-01 00:00:00,1,2013,3,1,0,5.0,14.0,4.0,12.0,200.0,85.0,-0.5,1024.5,-21.4,0.0,NNW,5.7,Nongzhanguan
2013-03-01 01:00:00,2,2013,3,1,1,8.0,12.0,6.0,14.0,200.0,84.0,-0.7,1025.1,-22.1,0.0,NW,3.9,Nongzhanguan
2013-03-01 02:00:00,3,2013,3,1,2,3.0,6.0,5.0,14.0,200.0,83.0,-1.2,1025.3,-24.6,0.0,NNW,5.3,Nongzhanguan
2013-03-01 03:00:00,4,2013,3,1,3,5.0,5.0,5.0,14.0,200.0,84.0,-1.4,1026.2,-25.5,0.0,N,4.9,Nongzhanguan
2013-03-01 04:00:00,5,2013,3,1,4,5.0,5.0,6.0,21.0,200.0,77.0,-1.9,1027.1,-24.5,0.0,NNW,3.2,Nongzhanguan


#### 3) Removing unwanted columns

Since we are intrested in time-series analysis of air pollutants We should remove unwanted columns.
The six major air-pollutants are:
1) PM2.5
2) PM10
3) SO2
4) NO2
5) CO
6) O3
<br>Rest of the columns need to be ommited


In [61]:
wanted_col = ['PM2.5','PM10','SO2','NO2','CO','O3']
df = df[wanted_col]

In [62]:
df.head()

,PM2.5,PM10,SO2,NO2,CO,O3
Date-Time,,,,,,
2013-03-01 00:00:00,5.0,14.0,4.0,12.0,200.0,85.0
2013-03-01 01:00:00,8.0,12.0,6.0,14.0,200.0,84.0
2013-03-01 02:00:00,3.0,6.0,5.0,14.0,200.0,83.0
2013-03-01 03:00:00,5.0,5.0,5.0,14.0,200.0,84.0
2013-03-01 04:00:00,5.0,5.0,6.0,21.0,200.0,77.0


#### 4) Handling Null values

In [63]:
df.info()

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 35064 entries, 2013-03-01 00:00:00 to 2017-02-28 23:00:00
Data columns (total 6 columns):
PM2.5    34436 non-null float64
PM10     34624 non-null float64
SO2      34618 non-null float64
NO2      34372 non-null float64
CO       33858 non-null float64
O3       34558 non-null float64
dtypes: float64(6)
memory usage: 1.9 MB
